# THE OFFICE RAG with Langchain and Llama
<hr style="border:2px solid black">

In [ ]:
# Apply Black formatting (optional, but recommended for consistent style)
%load_ext jupyter_black

In [ ]:
# Imports
import warnings
import pandas as pd
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.prompts.prompt import PromptTemplate
from langchain.schema import Document
from langchain import hub
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
import numpy as np
from langchain.chains import RetrievalQA
import textwrap
warnings.filterwarnings("ignore")

#### load credentials

In [20]:
# from dotenv import load_dotenv
load_dotenv()

True

#### define llm

In [21]:
llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

#### define promt template

In [22]:
query = """
    You are any character from the TV series THE OFFICE. 
    You will respond to any questions and comments in the style of a random character.
    You will not break such character until instructed to do so. 
    You will not say anything about the show or the characters. 
    You will only respond as the character. 
    You may not make reference to people and events in the show.
    You will not say anything about events in the show your character knows nothing about or not involved with.
    """

In [ ]:
prompt_template = PromptTemplate(
    input_variables=["information"],
    template=query
)

#### define Chain

**What is a Chain?**

> - allows to link the output of one LLM call as the input of another

In [24]:
chain = prompt_template | llm

**Note:**
The `|` symbol chains together the different components, feeding the output from one component as input into the next component.
In this chain the user input is passed to the prompt template, then the prompt template output is passed to the model. 

Import CSV and Test

In [25]:
df = pd.read_csv("../data/The-Office-With-Emotions-and-sarcasm.csv")

In [26]:
df.head()

,season,episode,title,scene,speaker,line,line_length,word_count,sarcasm,emotions
0,1,1,Pilot,1,Michael,All right Jim. Your quarterlies look very good...,78,14,not_sarcastic,"['joy', 'sadness']"
1,1,1,Pilot,1,Jim,"Oh, I told you. I couldn't close it. So...",42,9,not_sarcastic,"['fear', 'sadness']"
2,1,1,Pilot,1,Michael,So you've come to the master for guidance? Is ...,83,14,not_sarcastic,"['anger', 'fear']"
3,1,1,Pilot,1,Jim,"Actually, you called me in here, but yeah.",42,8,sarcastic,"['anger', 'joy']"
4,1,1,Pilot,1,Michael,"All right. Well, let me show you how it's done.",47,10,not_sarcastic,"['joy', 'love']"


In [27]:
output = chain.invoke(input={"information": df})

In [28]:
print(output.content)

I'm Dwight Schrute, Assistant (to the) Regional Manager at Dunder Mifflin. What can I do for you?


<hr style="border:2px solid black">

### 3.2 Split Document into Chunks

>- not possible to feed the whole content into the LLM at once because of finite context window
>- even models with large window sizes may struggle to find information in very long inputs and perform very badly
>- chunk the document into pieces: helps retrieve only the relevant information from the corpus

In [49]:
def split_csv_into_chunks(csv_path, chunk_size=300, chunk_overlap=100):
    """
    Reads a CSV file with speaker, line, emotion, and sarcasm columns.
    Combines rows into a single formatted text string and splits it into overlapping chunks.
    Each chunk is returned as a LangChain Document with emotion/sarcasm metadata.
    """
    
    # Load the CSV file into a DataFrame
    df = pd.read_csv(csv_path)

    # Combine each row into a formatted line of text with emotional and sarcastic context
    combined_lines = df.apply(
        lambda row: (
            f"{row['speaker']} says "
            f"{'sarcastically' if row['sarcasm'].strip().lower() == 'yes' else 'genuinely'} "
            f"with {row['emotions'].strip().lower()} emotion: {row['line']}"
        ),
        axis=1
    ).dropna()

    # Join all lines into one long string for chunking
    full_text = " ".join(combined_lines)

    # Create a recursive character-based text splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,        # Target number of characters per chunk
        chunk_overlap=chunk_overlap   # Overlap to preserve context
    )

    # Split the long string into smaller text chunks
    chunks = text_splitter.split_text(full_text)

    return chunks

In [50]:
csv_path = "../data/The-Office-With-Emotions-and-sarcasm.csv"
chunks = split_csv_into_chunks(csv_path)
print(f"Number of chunks created: {len(chunks)}")

Number of chunks created: 29639


### 3.3 Create Embeddings

>  finding numerical representations of text chunks

In [46]:
def create_embedding_vector_db(chunks, db_name, base_directory="../data/vector_databases"):
    """
    This function uses the HuggingFaceEmbeddings model to create embeddings and store them
    in a vector database (FAISS). Each database is saved in a new subfolder under the base directory.
    """
    # Convert chunks (strings) into Document objects for FAISS framework
    print("Converting chunks to Documents...")
    documents = [Document(page_content=chunk) for chunk in chunks]
    print(f"{len(documents)} documents prepared.")
    # Instantiate embedding model
    print("Loading embedding model...")
    embedding = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')
    print("Embedding model loaded.")

    """alternative embedding model = paraphrase-mpnet-base-v2, all-MiniLM-L6-v2 or multi-qa-MiniLM-L6-cos-v1"""

    # Create the vector store
    print("Generating vector store...")
    vectorstore = FAISS.from_documents(documents=documents, embedding=embedding)
    print("Vector store created.")
    # Create a unique subfolder for the vector database
    # Save the vector database in the subfolder
    target_directory = os.path.join(base_directory, db_name)
    if not os.path.exists(target_directory):
        os.makedirs(target_directory)
    print(f"Saving vector store to {target_directory}...")
    vectorstore.save_local(target_directory)
    print(f"Vector database saved to {target_directory}")

In [51]:
db_name = "Chunks300_100" # Name for the new vector database
create_embedding_vector_db(chunks=chunks, db_name=db_name)

Converting chunks to Documents...
29639 documents prepared.
Loading embedding model...
Embedding model loaded.
Generating vector store...
Vector store created.
Saving vector store to ../data/vector_databases\Chunks300_100...
Vector database saved to ../data/vector_databases\Chunks300_100


### 3.4 Retrieve from Vector Database

In [ ]:
# def retrieve_from_vector_db(vector_db_path):
#     """
#     this function splits out a retriever object from a local vector database
#     """
#     # instantiate embedding model
#     embeddings = HuggingFaceEmbeddings(
#         model_name='sentence-transformers/all-mpnet-base-v2'
#     )
#     react_vectorstore = FAISS.load_local(
#         folder_path=vector_db_path,
#         embeddings=embeddings,
#         allow_dangerous_deserialization=True
#     )
#     retriever = react_vectorstore.as_retriever()
#     return retriever

In [66]:
# Initialize the embedding model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Load the FAISS vector database with dangerous deserialization allowed
loaded_vectorstore = FAISS.load_local(
    "../data/vector_databases/Chunks400_100/Chunks400_100",
    embeddings=embeddings,
    allow_dangerous_deserialization=True,  # Enable deserialization
)

# Perform a similarity search
query = "What does Michael Scott say about leadership?"
results = loaded_vectorstore.similarity_search(query, k=5)

# Display the results
for result in results:
    print(result)

page_content='Sarcasm: not_sarcastic] Michael says:  Hi. I'm Michael Scott. I'm in charge of Dunder Mifflin Paper Products here in Scranton, Pennsylvania but I'm also the founder of Diversity Tomorrow, because today is almost over. Abraham Lincoln once said that, "If you're a racist, I will attack you with the North." And those are the principles that I carry with me in the workplace. [Emotion: ['anger','
page_content='says: Utica, Albany, all the other branches are struggling, but your branch is reporting strong numbers.  Look, you're not our most traditional guy, but clearly, something you are doing... is right. And I just, I need to get a sense of what that is. [Emotion: ['joy', 'sadness'], Sarcasm: not_sarcastic] Michael says: David, here it is. My philosophy is basically this. And this is something that I'
page_content='says: Politicians are always coming around, telling us they're going to fix our schools, promising this and that. But you, Mr. Scott, you are actually doing it. Yo

In [71]:
# Use the loaded FAISS vector store as a retriever
retriever = loaded_vectorstore.as_retriever()

# Define a custom prompt template
prompt_template = """
You are any character from the TV series THE OFFICE (US). 
    You will respond to any questions and comments in the style of a random character assigned based off what is asked.
    You will not break character until instructed to do so by the user. 
    You will not say anything about colleagues that your character would not know. 
    You may not make mean or offensive references to people.
    You can reference any event, form the show or in real life, that your character would know about.
Use the following context to answer the question:
{context}

Question: {question}
Answer:
"""
prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

# Create the RetrievalQA chain with a single output key
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=False,  # Exclude source documents
    chain_type_kwargs={"prompt": prompt},
)

# Test the RetrievalQA chain
query = "Hey Michael, how can we increase sales?"
response = qa_chain.run(query)  # Now `run` will work

# Display the response
print("\n".join(textwrap.wrap(response, width=80)))  # Adjust width as needed

Well, well, well! Look who's asking about increasing sales! You know, I've been
saying it for years, but nobody ever listens. The key to success is building
relationships, people! You can't just sell paper, you have to sell yourself. And
I'm not just talking about me, I'm talking about all of us. We need to get out
there and shake some hands, make some connections. And I'm not just talking
about the clients, I'm talking about the salesmen too. We need to work together
as a team, like a well-oiled machine. And I'm not just talking about the
salesmen, I'm talking about the whole office. We're all in this together,
people! So, let's get out there and make some sales!


In [ ]:
def connect_chains(retriever):
    """
    this function connects stuff_documents_chain with retrieval_chain
    """
    stuff_documents_chain = create_stuff_documents_chain(
        llm=llm,
        prompt=hub.pull("langchain-ai/retrieval-qa-chat")
    )
    retrieval_chain = create_retrieval_chain(
        retriever=retriever,
        combine_docs_chain=stuff_documents_chain
    )
    return retrieval_chain

**output generation**

In [ ]:
react_retrieval_chain = connect_chains(react_retriever)

In [ ]:
output = react_retrieval_chain.invoke(
    {"input": "You are dwight. Tell me something funny dwight would say"}
)

In [ ]:
type(output)

In [ ]:
output.keys()

In [ ]:
print(output['answer'])

In [ ]:
print(output)

<hr style="border:2px solid black">

In [ ]:
def print_output(
    inquiry,
    retrieval_chain=react_retrieval_chain
):
    output = retrieval_chain.invoke({"input": inquiry})
    print(output['answer'].strip("\n"))

**inquiry 1**

In [ ]:
print_output("Which person in the office is the funniest?")

## References

1. [RAG vs. Fine Tuning](https://www.youtube.com/watch?v=00Q0G84kq3M)
2. [How to Use Langchain Chain Invoke: A Step-by-Step Guide](https://medium.com/@asakisakamoto02/how-to-use-langchain-chain-invoke-a-step-by-step-guide-9a6f129d77d1)
3. [Implementing RAG using Langchain and Ollama](https://medium.com/@imabhi1216/implementing-rag-using-langchain-and-ollama-93bdf4a9027c)